In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [6]:
url_df = pd.read_csv('../data/url/2023_url.csv')

In [7]:
url_df.head()

,game,url
0,The Legend of Zelda: Tears of the Kingdom,https://www.metacritic.com/game/switch/the-leg...
1,Metroid Prime Remastered,https://www.metacritic.com/game/switch/metroid...
2,The Witcher 3: Wild Hunt - Complete Edition,https://www.metacritic.com/game/xbox-series-x/...
3,Tetris Effect: Connected,https://www.metacritic.com/game/playstation-5/...
4,Resident Evil 4,https://www.metacritic.com/game/playstation-5/...


In [8]:
df_user = pd.DataFrame(columns=['user_name', 'game']) ##############

for i,name in zip(url_df['url'], url_df['game']):
    print(name)
    # url 가져오기
    url = i + '/user-reviews?sort-by=most-helpful&num_items=100'

    #print(url)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # 페이지 수 확인
    last_page_element = soup.find("li", class_="last_page")
    if last_page_element is not None:
        last_page = int(last_page_element.find("a").text)
    else:
        last_page = 1

    users = []
    for i in range(last_page):
        page_url = url + '#&page=' + str(i)

        response = requests.get(page_url, headers=headers)
        html = response.text

        # BeautifulSoup을 사용하여 HTML 문서 파싱
        soup = BeautifulSoup(html, 'html.parser')

        # 사용자 리뷰 엘리먼트를 찾아서 출력
        user = soup.find_all('div', class_='name')
        
        
        for u in user: users.append(u.get_text(strip=True))

    cnt=0
    for u in users:
        if cnt>=10: break
        user_url = 'https://www.metacritic.com/user/' + u
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }

        response = requests.get(user_url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")

        # 사용자 리뷰 엘리먼트를 찾아서 출력
        title_info = soup.find("ol", class_="reviews user_reviews")

        # title_info에 어떠한 정보라도 있어야 find_all 함수를 사용할 수 있다.
        if title_info is not None:
            titles = title_info.find_all("div", class_="product_title")

            titles_arr = []
            for t in titles:
                titles_arr.append(t.get_text(strip=True))

            if len(titles_arr)>=3:
                user_tmp = pd.DataFrame(columns=['user_name', 'game'])
                user_tmp.loc[0] = [u, titles_arr]
                df_user = pd.concat([df_user, user_tmp])
                cnt += 1
    
    print(df_user)
    print('******************************************')
    print('******************************************')           
        

         user_name                                               game
0   PotatoMaster69  [The Legend of Zelda: Tears of the Kingdom, Me...
0        Andresfrq  [The Legend of Zelda: Tears of the Kingdom, Ki...
0     spencerplewe  [The Legend of Zelda: Tears of the Kingdom, Th...
0  Epic_Gamerrater  [The Legend of Zelda: Tears of the Kingdom, Ul...
0        Linebeckk  [The Legend of Zelda: Tears of the Kingdom, Po...
0       Hippystank  [For Honor, Hogwarts Legacy, Diablo IV, The Le...
0        Starazor0  [The Legend of Zelda: Tears of the Kingdom, Ma...
0           Masigh  [Borderlands 3, The Legend of Zelda: Tears of ...
0    dannysgamebox  [The Legend of Zelda: Tears of the Kingdom, Ad...
0           ZYMROD  [The Legend of Zelda: Tears of the Kingdom, St...
******************************************
******************************************
         user_name                                               game
0   PotatoMaster69  [The Legend of Zelda: Tears of the Kingdom, Me...
0   

ConnectionError: HTTPSConnectionPool(host='www.metacritic.com', port=443): Max retries exceeded with url: /user/rthn (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7efccf730a30>: Failed to establish a new connection: [Errno 110] Connection timed out'))

In [9]:
df_user

,user_name,game
0,PotatoMaster69,"[The Legend of Zelda: Tears of the Kingdom, Me..."
0,Andresfrq,"[The Legend of Zelda: Tears of the Kingdom, Ki..."
0,spencerplewe,"[The Legend of Zelda: Tears of the Kingdom, Th..."
0,Epic_Gamerrater,"[The Legend of Zelda: Tears of the Kingdom, Ul..."
0,Linebeckk,"[The Legend of Zelda: Tears of the Kingdom, Po..."
...,...,...
0,Yexela,"[Trepang2, Nova Lands, Jack Move]"
0,vgmkyle,"[AEW: Fight Forever, AEW: Fight Forever, AEW: ..."
0,S3XU4LH34L1NG,"[Tunic, Wasteland 3, No Man's Sky, The Legend ..."
0,matee75,"[Shadow Warrior 3, Resident Evil 4, Star Wars ..."


In [10]:
df_user_2 = df_user.drop_duplicates(subset='user_name')

In [11]:
df_user_2

,user_name,game
0,PotatoMaster69,"[The Legend of Zelda: Tears of the Kingdom, Me..."
0,Andresfrq,"[The Legend of Zelda: Tears of the Kingdom, Ki..."
0,spencerplewe,"[The Legend of Zelda: Tears of the Kingdom, Th..."
0,Epic_Gamerrater,"[The Legend of Zelda: Tears of the Kingdom, Ul..."
0,Linebeckk,"[The Legend of Zelda: Tears of the Kingdom, Po..."
...,...,...
0,Wretic,"[Trepang2, The Legend of Zelda: Tears of the K..."
0,Qwest1991,"[Daydream: Forgotten Sorrow, Trepang2, Diablo ..."
0,Yexela,"[Trepang2, Nova Lands, Jack Move]"
0,S3XU4LH34L1NG,"[Tunic, Wasteland 3, No Man's Sky, The Legend ..."


In [27]:
# 새로운 데이터프레임 생성
new_df = pd.DataFrame(columns=['user_name', 'game'])

# 기존 데이터프레임을 행 별로 반복하며 변경
for index, row in df_user_2.iterrows():
    user_name = row['user_name']
    games = row['game']
    
    for game in games:
        new_row = {
            'user_name': user_name,
            'game': game
        }
        new_row_df = pd.DataFrame([new_row])
        new_df = pd.concat([new_df, new_row_df])

In [28]:
new_df

,user_name,game
0,PotatoMaster69,The Legend of Zelda: Tears of the Kingdom
0,PotatoMaster69,Metroid Prime Remastered
0,PotatoMaster69,Hi-Fi Rush
0,PotatoMaster69,Halo Infinite
0,PotatoMaster69,The Last of Us Part I
...,...,...
0,S3XU4LH34L1NG,Weird West
0,S3XU4LH34L1NG,A Plague Tale: Requiem
0,Sunna,Saints Row: The Third
0,Sunna,Wo Long: Fallen Dynasty


In [29]:
# 중복 행 제거
new_df = new_df.drop_duplicates(['user_name', 'game'])

# 인덱스 재정렬
new_df = new_df.reset_index(drop=True)

In [30]:
new_df

,user_name,game
0,PotatoMaster69,The Legend of Zelda: Tears of the Kingdom
1,PotatoMaster69,Metroid Prime Remastered
2,PotatoMaster69,Hi-Fi Rush
3,PotatoMaster69,Halo Infinite
4,PotatoMaster69,The Last of Us Part I
...,...,...
4880,S3XU4LH34L1NG,Weird West
4881,S3XU4LH34L1NG,A Plague Tale: Requiem
4882,Sunna,Saints Row: The Third
4883,Sunna,Wo Long: Fallen Dynasty


In [31]:
new_df.to_csv('./2023_user_log_sample.csv', index=False)